In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
import os
from torch.cuda.amp import GradScaler, autocast
import timm  # Import timm for model loading
import random
import string
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap
import umap.umap_ as umap
import matplotlib.pyplot as plt
import numpy as np

# Task-Specific Augmentations for Non-Surgical Images
class TaskSpecificAugmentations:
    def __init__(self, watermark_text=None, watermark_size_range=(100, 100), random_text_length=8, font_path=None):
        self.watermark_text = watermark_text
        self.watermark_size_range = watermark_size_range
        self.random_text_length = random_text_length
        self.font_path = font_path or "/usr/share/fonts/truetype/Roboto-Regular.ttf"

    def generate_random_text(self):
        return ''.join(random.choices(string.ascii_uppercase + string.digits, k=self.random_text_length))

    def add_watermark(self, image):
        draw = ImageDraw.Draw(image)
        width, height = image.size
        text = self.watermark_text if self.watermark_text else self.generate_random_text()

        font_size = random.randint(*self.watermark_size_range)
        font = ImageFont.truetype(self.font_path, font_size)  # Use the specified TrueType font
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        position = ((width - text_width) // 2, (height - text_height) // 2)

        draw.text(position, text, fill=(255, 255, 255), font=font)
        return image

    def make_non_surgical(self, image):
        return image.convert("L").convert("RGB")

def random_non_surgical_augmentation(img):
    if random.random() > 0.5:
        return task_specific_augmentations.make_non_surgical(img)
    else:
        return task_specific_augmentations.add_watermark(img)

# Dataset class for loading images and applying augmentations
class UnannotatedImagesDataset(Dataset):
    def __init__(self, image_dir, transform_blur=None, transform_sharp=None, transform_non_surgical=None):
        self.image_dir = image_dir
        self.transform_blur = transform_blur
        self.transform_sharp = transform_sharp
        self.transform_non_surgical = transform_non_surgical
        self.image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('jpg', 'jpeg', 'png'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')

        # Apply the specific transformations for blurred and sharp images
        image_blur = self.transform_blur(image) if self.transform_blur else transforms.ToTensor()(image)
        image_sharp = self.transform_sharp(image) if self.transform_sharp else transforms.ToTensor()(image)
        image_non_surgical = self.transform_non_surgical(image) if self.transform_non_surgical else transforms.ToTensor()(image)

        image_blur = transforms.Resize((224,224))(image_blur)
        image_sharp = transforms.Resize((224,224))(image_sharp)
        image_non_surgical = transforms.Resize((224,224))(image_non_surgical)

        return image_blur, image_sharp, image_non_surgical

# File paths and transformations
newimages_image_dir = '/path_to_unlabelled_frames'

# Transformation for Blurry (Uninformative) Images
transform_blur = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur(7),  # Apply Gaussian blur
    transforms.ToTensor(),
    transforms.Lambda(lambda img: F.interpolate(img.unsqueeze(0), scale_factor=0.5, mode='bilinear', align_corners=False).squeeze(0)),  # Downsample and then upsample to simulate low resolution
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Transformation for Sharp (Informative) Images
transform_sharp = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

task_specific_augmentations = TaskSpecificAugmentations()

# Transformation for Non-Surgical Images with Random Grayscale or Watermark
transform_non_surgical = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda img: random_non_surgical_augmentation(img)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the large unannotated dataset with task-specific augmentations
newimages_dataset = UnannotatedImagesDataset(
    newimages_image_dir,
    transform_blur=transform_blur,
    transform_sharp=transform_sharp,
    transform_non_surgical=transform_non_surgical
)
contrastive_dataloader = DataLoader(newimages_dataset, batch_size=64, shuffle=True, num_workers=4)

# SimCLR Projection Head
class SimCLRHead(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_dim=2048):
        super(SimCLRHead, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),  # Increase dimensionality to hidden_dim (e.g., 2048)
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim),  # Reduce to the desired output dimension (e.g., 128)
        )

    def forward(self, x):
        x = self.mlp(x)
        return F.normalize(x, dim=-1)  # L2 normalization to ensure unit-length vectors

# Using EfficientNetV2-S model
base_model = timm.create_model('tf_efficientnetv2_s', pretrained=True)

# Get the number of input features for the final fully connected layer
num_ftrs = base_model.classifier.in_features

# Replace the fully connected layer with an identity layer
base_model.classifier = nn.Identity()

# Combine the backbone with the SimCLR head
model = nn.Sequential(
    base_model,
    SimCLRHead(num_ftrs, 128)  # 128 is the dimensionality of the projection space
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def reverse_triplet_loss(blur_out, sharp_out, non_surgical_out, temperature=0.1):
    # Calculate similarities between different augmentations of the same image
    sim_blur_sharp = F.cosine_similarity(blur_out, sharp_out, dim=-1)
    sim_sharp_non_surgical = F.cosine_similarity(sharp_out, non_surgical_out, dim=-1)
    sim_blur_non_surgical = F.cosine_similarity(blur_out, non_surgical_out, dim=-1)

    # We want to minimize these similarities (i.e., push them apart)
    reverse_loss_blur_sharp = -sim_blur_sharp.mean()
    reverse_loss_sharp_non_surgical = -sim_sharp_non_surgical.mean()
    reverse_loss_blur_non_surgical = -sim_blur_non_surgical.mean()

    # Combine the reversed losses into a total loss
    total_reverse_loss = (reverse_loss_blur_sharp + reverse_loss_sharp_non_surgical + reverse_loss_blur_non_surgical) / 3.0

    return total_reverse_loss

# Function to save checkpoints
def save_checkpoint(state, filename='/path_to_checkpoint_contrastive.pth.tar'):
    torch.save(state, filename)

import seaborn as sns
import matplotlib.pyplot as plt

def visualize_embeddings(model, dataloader, epoch):
    model.eval()
    embeddings = []
    labels = []

    with torch.no_grad():
        for (blur_images, sharp_images, non_surgical_images) in dataloader:
            blur_images = blur_images.to(device)
            sharp_images = sharp_images.to(device)
            non_surgical_images = non_surgical_images.to(device)

            blur_out = model(blur_images).cpu().numpy()
            sharp_out = model(sharp_images).cpu().numpy()
            non_surgical_out = model(non_surgical_images).cpu().numpy()

            embeddings.append(blur_out)
            embeddings.append(sharp_out)
            embeddings.append(non_surgical_out)

            labels.extend([0] * blur_out.shape[0])
            labels.extend([1] * sharp_out.shape[0])
            labels.extend([2] * non_surgical_out.shape[0])

    embeddings = np.concatenate(embeddings, axis=0)
    labels = np.array(labels)

    palette = sns.color_palette("Set2", 3)  # Set2 is a palette with soft, yet distinct colors

    # t-SNE
    tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
    embeddings_tsne = tsne.fit_transform(embeddings)

    # PCA
    pca = PCA(n_components=2)
    embeddings_pca = pca.fit_transform(embeddings)

    # UMAP
    reducer = umap.UMAP()
    embeddings_umap = reducer.fit_transform(embeddings)

    plt.figure(figsize=(20, 6))

    # t-SNE Plot
    plt.subplot(1, 3, 1)
    plt.scatter(embeddings_tsne[labels == 0, 0], embeddings_tsne[labels == 0, 1], color=palette[0], label='Blurred', alpha=0.7, edgecolor='k')
    plt.scatter(embeddings_tsne[labels == 1, 0], embeddings_tsne[labels == 1, 1], color=palette[1], label='Sharp', alpha=0.7, edgecolor='k')
    plt.scatter(embeddings_tsne[labels == 2, 0], embeddings_tsne[labels == 2, 1], color=palette[2], label='Non-Surgical', alpha=0.7, edgecolor='k')
    plt.legend()
    plt.title(f't-SNE (Epoch {epoch})')
    plt.grid(True)

    # PCA Plot
    plt.subplot(1, 3, 2)
    plt.scatter(embeddings_pca[labels == 0, 0], embeddings_pca[labels == 0, 1], color=palette[0], label='Blurred', alpha=0.7, edgecolor='k')
    plt.scatter(embeddings_pca[labels == 1, 0], embeddings_pca[labels == 1, 1], color=palette[1], label='Sharp', alpha=0.7, edgecolor='k')
    plt.scatter(embeddings_pca[labels == 2, 0], embeddings_pca[labels == 2, 1], color=palette[2], label='Non-Surgical', alpha=0.7, edgecolor='k')
    plt.legend()
    plt.title(f'PCA (Epoch {epoch})')
    plt.grid(True)

    # UMAP Plot
    plt.subplot(1, 3, 3)
    plt.scatter(embeddings_umap[labels == 0, 0], embeddings_umap[labels == 0, 1], color=palette[0], label='Blurred', alpha=0.7, edgecolor='k')
    plt.scatter(embeddings_umap[labels == 1, 0], embeddings_umap[labels == 1, 1], color=palette[1], label='Sharp', alpha=0.7, edgecolor='k')
    plt.scatter(embeddings_umap[labels == 2, 0], embeddings_umap[labels == 2, 1], color=palette[2], label='Non-Surgical', alpha=0.7, edgecolor='k')
    plt.legend()
    plt.title(f'UMAP (Epoch {epoch})')
    plt.grid(True)

    plt.show()


# Training setup
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scaler = GradScaler()  # Initialize the gradient scaler for mixed precision

# Early stopping setup
patience = 3
best_loss = float('inf')
epochs_no_improve = 0

# Training loop
for epoch in range(30):  # Number of epochs
    model.train()
    epoch_loss = 0.0  # Initialize epoch loss

    progress_bar = tqdm(enumerate(contrastive_dataloader), total=len(contrastive_dataloader), desc=f"Epoch [{epoch+1}/30]")
    for batch_idx, (blur_images, sharp_images, non_surgical_images) in progress_bar:
        blur_images = blur_images.to(device)
        sharp_images = sharp_images.to(device)
        non_surgical_images = non_surgical_images.to(device)

        optimizer.zero_grad()  # Reset gradients at the start of each batch

        with autocast():  # Enable mixed precision training
            blur_out = model(blur_images)
            sharp_out = model(sharp_images)
            non_surgical_out = model(non_surgical_images)

            # Use the contrastive loss
            loss = reverse_triplet_loss(blur_out, sharp_out, non_surgical_out, temperature=0.1)

        scaler.scale(loss).backward()  # Scale the loss and perform backpropagation

        scaler.step(optimizer)  # Update model parameters
        scaler.update()  # Update the scaler

        epoch_loss += loss.item()  # Accumulate batch loss

        # Update progress bar with loss
        progress_bar.set_postfix(loss=loss.item())

    avg_epoch_loss = epoch_loss / len(contrastive_dataloader)
    print(f"Epoch [{epoch+1}/30], Average Loss: {avg_epoch_loss:.4f}")

    # Visualization after each epoch
    visualize_embeddings(model, contrastive_dataloader, epoch + 1)

    # Early stopping and best model checkpointing
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        epochs_no_improve = 0

        # Save the best model checkpoint
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_loss': best_loss,
            'optimizer': optimizer.state_dict(),
        }, filename='/path_to_best_model_checkpoint.pth.tar')
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print("Early stopping triggered.")
        break

    # Save regular checkpoints every few epochs
    if (epoch + 1) % 2 == 0:  # For example, save every 2 epochs
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_loss': best_loss,
            'optimizer': optimizer.state_dict(),
        }, filename=f'/path_to_checkpoint_epoch_{epoch+1}.pth.tar')

# Save the final model
torch.save(model.state_dict(), '/path_to_pretrained_model_contrastive.pth')
